In [15]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pickle



In [16]:
# Load the dataset
df = pd.read_csv('2025_train.csv', low_memory=False)


df['year_index'] = pd.Series(df['academicYear']).rank(method='dense').astype(int)

# This is needed to identify the original rows after scaling
# Will be used to analyze worst predictions
df['idOSYM_flag'] = df['idOSYM']

# Set academicYear as index if needed
df.set_index('year_index', inplace=True)

df_2019 = df.loc[1]  # Based on your date format
print("DataFrame with year index 2019:" ,df_2019.head())

DataFrame with year index 2019:             academicYear  universityName  faculty  departmentName  \
year_index                                                          
1                   2019               0      201              49   
1                   2019               0      290              80   
1                   2019               0      292             126   
1                   2019               0      201             133   
1                   2019               0      201             152   

                 idOSYM  scholarshipRate  quota  occupiedSlots  tuitionFee  \
year_index                                                                   
1           106510077.0                0   62.0           62.0         0.0   
1           106510087.0                0   62.0           62.0         0.0   
1           106510088.0                0   52.0           52.0         0.0   
1           106510068.0                0   62.0           62.0         0.0   
1           1065

In [17]:
binary_features = [
    'universityType_devlet', 'universityType_vakıf', 'programType_DİL', 'programType_EA', 
    'programType_SAY', 'programType_SÖZ', 'language_Almanca', 'language_Arapça', 'language_Bulgarca',  
    'language_Ermenice', 'language_Fransızca', 'language_Korece', 'language_Lehçe', 'language_Rusça', 
    'language_Türkçe', 'language_Çince', 'language_İngilizce', 'language_İspanyolca', 'language_İtalyanca', 
    'baseRanking','idOSYM_flag'
]
print(df[binary_features].describe().round(3))
print(len(binary_features))



       universityType_devlet  universityType_vakıf  programType_DİL  \
count              62273.000             62273.000        62273.000   
mean                   0.602                 0.398            0.050   
std                    0.490                 0.490            0.218   
min                    0.000                 0.000            0.000   
25%                    0.000                 0.000            0.000   
50%                    1.000                 0.000            0.000   
75%                    1.000                 1.000            0.000   
max                    1.000                 1.000            1.000   

       programType_EA  programType_SAY  programType_SÖZ  language_Almanca  \
count       62273.000        62273.000        62273.000         62273.000   
mean            0.324            0.455            0.171             0.004   
std             0.468            0.498            0.376             0.063   
min             0.000            0.000            0.

In [18]:
numeric_features = []
for feature in df.columns:
    if feature not in binary_features:
        numeric_features.append(feature)
        
print(len(numeric_features))

42


In [19]:
skew_values = df[numeric_features].skew()

skew_features = skew_values[abs(skew_values) > 1.5].index.tolist()

print("Skewed Features:", skew_features)
print(len(skew_features))

continuous_features = [col for col in numeric_features if col not in skew_features]

print("Continous Features:", continuous_features)
print(len(continuous_features))

Skewed Features: ['quota', 'occupiedSlots', 'tuitionFee', 'profCount', 'assoCount', 'docCount', 'revenue', 'totalPreference', 'top1PreferenceRatio', 'avgAdmittedStudentPrefOrder', 'top10AdmittedRatio', 'admittedTotalPref', 'admittedTotalDepartmentPref', 'currentStudentCount', 'totalForeignStudents']
15
Continous Features: ['academicYear', 'universityName', 'faculty', 'departmentName', 'idOSYM', 'scholarshipRate', 'universityLocation', 'universityRegion', 'topRanking', 'avgAdmissionRanking', 'baseAdmissionRanking', 'stdDeviationStudents', 'outOfCityStudentRate', 'avgOrderofPreference', 'top1AdmittedRatio', 'top3AdmittedRatio', 'baseScore', 'topScore', 'totalStudentNumber', 'Urap_Rank', 'Urap_Score', 'avg_monthly_income_group', 'Time_for_employment', 'employment_rate', 'base_salary_by_year', 'inflation_by_year', 'growth_by_year']
27


In [20]:
scaler_standard = StandardScaler()

# Step 1: Apply log transformation to skewed features
df[skew_features] = df[skew_features] + 0.1  # Add small constant to avoid log(0)
df[skew_features] = np.log1p(df[skew_features])

# Step 2: Combine all features into a single dataset
numeric_features = continuous_features + skew_features

# Step 3: Change to int
df[numeric_features] = df[numeric_features].astype(float)

# Step 3: Apply the same scaling to all features
preprocessor = ColumnTransformer(
    transformers=[
        ('all', scaler_standard, numeric_features),
        ('binary', 'passthrough', binary_features)
    ]
)

# Fit and transform the data
processed_data = preprocessor.fit_transform(df)

# Convert the result back to a DataFrame
processed_df = pd.DataFrame(processed_data, columns=numeric_features+binary_features)

# Save the fitted preprocessor as a pickle file
with open("scaler.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

# Print descriptive statistics
print(processed_df.describe().round(3))

       academicYear  universityName    faculty  departmentName     idOSYM  \
count     62273.000       62273.000  62273.000       62273.000  62273.000   
mean         -0.000          -0.000      0.000           0.000     -0.000   
std           1.000           1.000      1.000           1.000      1.000   
min          -1.679          -1.759     -2.304          -1.916     -0.930   
25%          -0.649          -0.912     -1.099          -0.859     -0.833   
50%          -0.134           0.052     -0.138          -0.066     -0.735   
75%           0.897           0.915      0.651           0.875      1.195   
max           1.412           1.629      2.186           1.739      1.352   

       scholarshipRate  universityLocation  universityRegion  topRanking  \
count        62273.000           62273.000         62273.000   62273.000   
mean            -0.000              -0.000             0.000      -0.000   
std              1.000               1.000             1.000       1.000   
mi

In [22]:
processed_df.to_csv('train.csv', index=False)

In [23]:
## Print data types
print(processed_df.dtypes)

academicYear           float64
universityName         float64
faculty                float64
departmentName         float64
idOSYM                 float64
                        ...   
language_İngilizce     float64
language_İspanyolca    float64
language_İtalyanca     float64
baseRanking            float64
idOSYM_flag            float64
Length: 63, dtype: object
